# Data Rangers: Prueba de Ingenieria de Datos Pragma

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from utils.functions import list_datafile, batch_processing
from pipeline.transforms import add_timestamp, parse_date
from pipeline.load import row_loads
from database.database import Database

from utils.logs import PipelineLogs

load_dotenv(dotenv_path='./db.env')

True

In [2]:
logs = PipelineLogs("Loguiador", 'logs.log', True, True).pipeline_logs()

In [3]:
logs.handlers

[<RotatingFileHandler /home/fernando/Desktop/Prueba_Pragma/logs.log (NOTSET)>,
 <StreamHandler stdout (NOTSET)>]

In [4]:
logs.info("Informacion necesaria")

[LINE:1] #INFO     [2025-02-12 16:57:39,892] Informacion necesaria


In [ ]:
USER = os.getenv('DB_USER')
PWD = os.getenv('DB_PASSWORD')
HOST = os.getenv('DB_HOST')
DB = os.getenv('DATABASE_NAME')

CHUNKSIZE = 3

plsql_db = Database(HOST, USER, PWD, DB)

In [ ]:
pathlist = list_datafile(path='./data')
files = pathlist
for f in files:
    df_gen = batch_processing(f, chunks=CHUNKSIZE)
    for ch in df_gen:
        dfs:pd.DataFrame = ch
        df =(
            dfs.pipe(add_timestamp)
            .pipe(parse_date)
            )
        for row in dfs.itertuples(index=False):
            row, calendar_row = row_loads(row)
            plsql_db.insert(row, 'sales')
            print(f'-> inserting values in file {f}')

In [ ]:
row['price'] = 40
row

In [ ]:
sales = plsql_db.sales
engine = plsql_db.engine

In [ ]:
from sqlalchemy import and_

__columns = sales.columns
stmt = sales.select().where(and_(__columns.user_id == row['user_id'], __columns.timestamp == row['timestamp'], __columns.price == row['price']))

In [ ]:
result = plsql_db.__querys(stmt)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    values = result.fetchall()

In [ ]:
values[0][0]

In [ ]:
stmt = sales.update().where(__columns.unique_id == values[0][0]).values(row)

with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

In [ ]:
stmt = sales.select().where(and_(__columns.user_id == row['user_id'], __columns.timestamp == row['timestamp'], __columns.price == row['price']))
with engine.connect() as conn:
    result = conn.execute(stmt)
    values = result.fetchall()
    conn.commit()

values

In [ ]:
stmt = sales.update().values(row)

with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()